This notebook is need more than 32gb RAM memory to run

## 1. Import necessary packages

In [ ]:
!pip install catboost
!apt-get install unrar
!pip install  lightgbm

In [ ]:
env='kaggle'
if env=='colab':
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
import numpy as np # linear algebra
from tqdm.auto import tqdm
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

## 2. loading file

In [ ]:
#parameters
seed=41

#agg fun list
agg_lst=['sum', 'mean', 'max', 'min', 'std', 'var',  'sem',   'mad', 'prod']

# main features
time_cols=['hour', 'hour//2', 'hour//4', 'hour//6', 'hour//8', 'dayofweek', 'day','weekday', 'weekend',
    'dayofweekhour', 'daydayofweekhour', 'month']
main_cols=['access_success_rate', 'resource_utilition_rate', 'TA', 'bler', 'cqi', 'mcs', 'data_rate']
diff1_cols=[f'{i}_diff1' for i in main_cols]
diff2_cols=[f'{i}_diff2' for i in main_cols]
diff3_cols=[f'{i}_diff3' for i in main_cols]

sum_diff_cols=[f'{i}_sum_diff' for i in main_cols]
sum_ratio_cols=[f'{i}_sum_ratio' for i in main_cols]
sum_binary_cols=[f'{i}_sum_binary' for i in main_cols]

diff1_binary_cols=[f'{i}_diff1_binary' for i in main_cols]
diff2_binary_cols=[f'{i}_diff2_binary' for i in main_cols]
diff3_binary_cols=[f'{i}_diff3_binary' for i in main_cols]

ratio1_cols=[f'{i}_ratio1' for i in main_cols]
ratio2_cols=[f'{i}_ratio2' for i in main_cols]
ratio3_cols=[f'{i}_ratio3' for i in main_cols]


weight_mian_cols=[1.638127,  1.566788,  0.723693,  0.605481,  1.348831,  3.195837,
       30.105474]

In [ ]:
import glob


file_is_rar=False
if file_is_rar:
    !unrar x "/content/drive/MyDrive/network_faulty_analysis_data/imgs_2023050915314323740.rar" "/content/drive/MyDrive/network_faulty_analysis_data/train"
    !unrar x "/content/drive/MyDrive/network_faulty_analysis_data/validation_clean.rar" "/content/drive/MyDrive/network_faulty_analysis_data/val"

if env=='colab':
      path_train = "/content/drive/MyDrive/network_faulty_analysis_data/train_test_dataset_Fault Impact Analysis"
      path_test="/content/drive/MyDrive/network_faulty_analysis_data/val"
    
else:
   path_train= '/kaggle/input/new-update-5g/train_test_dataset_Fault Impact Analysis'
   path_test='/kaggle/input/fualty-analyst/validation_clean'
train_file = glob.glob(path_train+'/**')
test_file = glob.glob(path_test+'/**')

# sample submission path
if env=='colab':
    path_sub='/content/drive/MyDrive/network_faulty_analysis_data/SampleSubmission.csv'
else:
    path_sub='/kaggle/input/fualty-analyst/SampleSubmission (1).csv'

In [ ]:
def date_features(df):
    df['endTime']=pd.to_datetime(df['endTime'])
    df['hour']=(df['endTime'].dt.hour).astype(int)
    df['hour//2']=(df['hour']//2).astype(int)
    df['hour//4']=(df['hour']//4).astype(int)
    df['hour//6']=(df['hour']//6).astype(int)
    df['hour//8']=(df['hour']//8).astype(int)

    df['dayofweek']=(df['endTime'].dt.dayofweek).astype(int)
    df['day']=(df['endTime'].dt.day).astype(int)
    df['weekday']=(df['endTime'].dt.weekday).astype(int)
    df['weekend']=(df['dayofweek'].isin([6, 0])).astype(int)

    df['dayofweekhour']=df[['dayofweek', 'hour']].astype(str).apply(''.join, axis=1).astype(int)
    df['daydayofweekhour']=df[['day', 'dayofweek', 'hour']].astype(str).apply(''.join, axis=1).astype(int)
    df['month']=(df['endTime'].dt.month).astype(int)
    return df

In [ ]:
import pathlib
def reader(f):
    try:

        df = pd.read_csv(f)
        #get id
        df['id']=f.split('/')[-1].split('.')[0]
        df_full=df.copy()
        try:
            #to get the difference of (t+1) -(t-1)
            df_full[diff1_cols]=df_full[main_cols].diff(1)
            df_full[diff2_cols]=df_full[diff1_cols].diff(1)
            df_full[diff3_cols]=df_full[diff2_cols].diff(1)

            df_full[diff1_binary_cols]=(df_full[diff1_cols]<0).astype(int)
            df_full[diff2_binary_cols]=(df_full[diff2_cols]<0).astype(int)
            df_full[diff3_binary_cols]=(df_full[diff3_cols]<0).astype(int)

        except:
            pass

        try:
            #to get ratio of chnage on data rate of t+1) -(t)
            df_full[ratio1_cols]=df_full[main_cols]/df[main_cols].shift(1)
            df_full[ratio2_cols]=df_full[ratio1_cols]/df[ratio1_cols].shift(1)
            df_full[ratio3_cols]=df_full[ratio2_cols]/df[ratio2_cols].shift(1)

        except:
            pass

        try:
            #to get sum of t+1 - t-1 and t+1/t-1
            sum_diff_cols=[f'{i}_sum_diff' for i in main_cols]
            df_full[sum_diff_cols]=(df_full[diff1_cols]+df_full[diff2_cols]+df_full[diff3_cols]).values
        except:
            pass


        # temporary df to create label this df indict the first df when fualty duration great than zero
        temp=df[df['fault_duration']>0][:1]

        # creat label if data rate less than previous data rate 1 else 0
        temp['label']=(df[df['fault_duration']==0][-1:]['data_rate'].sum()>df[df['fault_duration']>0]['data_rate'][:1].sum()).astype(int)

        # to create the predictors features which are t-1
        df_full_before_f=df_full[df_full['fault_duration']==0]
        df_full_after_f=df_full[df_full['fault_duration']>0]

        # to assign X columns
        df=df_full_before_f[-1:]


        #to replace fault duration, relation and label values with t+1 values
        df[['endTime', 'fault_duration', 'relation', 'label']]=temp[['endTime','fault_duration', 'relation', 'label']].values


        # to take the t+1 time  values

        df[[f'{i}_diff1+1' for i in main_cols]]=df_full_after_f[:1][diff1_cols].values
        df[[f'{i}_diff2+1' for i in main_cols]]=df_full_after_f[:1][diff2_cols].values
        df[[f'{i}_diff3+1' for i in main_cols]]=df_full_after_f[:1][diff3_cols].values


        try:
            df[[f'{i}_ratio2+1' for i in main_cols]]=df_full_after_f[:1][ratio2_cols].values
            df[[f'{i}_ratio3+1' for i in main_cols]]=df_full_after_f[:1][ratio3_cols].values
            df[[f'{i}_ratio1+1' for i in main_cols]]=df_full_after_f[:1][ratio1_cols].values
        except:
            pass





        # simple weighted average
        try:
            weights1=pd.DataFrame([.3, .7]).values
            df[[f'{i}_diff1_roll_2' for i in main_cols]]=(df_full_before_f[diff1_cols][-2:].values*weights1).mean(axis=0)
            df[[f'{i}_diff2_roll_2' for i in main_cols]]=(df_full_before_f[diff2_cols][-2:].values*weights1).mean(axis=0)
            df[[f'{i}_diff3_roll_2' for i in main_cols]]=(df_full_before_f[diff3_cols][-2:].values*weights1).mean(axis=0)

            weights2=pd.DataFrame([0.2, .3, .5]).values
            df[[f'{i}_diff1_roll_3' for i in main_cols]]=(df_full_before_f[diff1_cols][-3:].values*weights2).mean(axis=0)
            df[[f'{i}_diff2_roll_3' for i in main_cols]]=(df_full_before_f[diff2_cols][-3:].values*weights2).mean(axis=0)
            df[[f'{i}_diff3_roll_3' for i in main_cols]]=(df_full_before_f[diff3_cols][-3:].values*weights2).mean(axis=0)


            weights3=pd.DataFrame([0.1, 0.2, .3, .4]).values
            df[[f'{i}_diff1_roll_4' for i in main_cols]]=(df_full_before_f[diff1_cols][-4:].values*weights3).mean(axis=0)
            df[[f'{i}_diff2_roll_4' for i in main_cols]]=(df_full_before_f[diff2_cols][-4:].values*weights3).mean(axis=0)
            df[[f'{i}_diff3_roll_4' for i in main_cols]]=(df_full_before_f[diff3_cols][-4:].values*weights3).mean(axis=0)

            weights5=pd.DataFrame([0.05, 0.15, 0.2, .25, .35]).values
            df[[f'{i}_diff1_roll_5' for i in main_cols]]=(df_full_before_f[diff1_cols][-5:].values*weights5).mean(axis=0)
            df[[f'{i}_diff2_roll_5' for i in main_cols]]=(df_full_before_f[diff2_cols][-5:].values*weights5).mean(axis=0)
            df[[f'{i}_diff3_roll_5' for i in main_cols]]=(df_full_before_f[diff3_cols][-5:].values*weights5).mean(axis=0)


            weights6=pd.DataFrame([0.05, 0.1, 0.15, 0.15, 0.2, 0.35]).values
            df[[f'{i}_diff1_roll_6' for i in main_cols]]=(df_full_before_f[diff1_cols][-6:].values*weights6).mean(axis=0)
            df[[f'{i}_diff2_roll_6' for i in main_cols]]=(df_full_before_f[diff2_cols][-6:].values*weights6).mean(axis=0)
            df[[f'{i}_diff3_roll_6' for i in main_cols]]=(df_full_before_f[diff3_cols][-6:].values*weights6).mean(axis=0)


            weights7=pd.DataFrame([0.1, 0.1, 0.1, 0.13, 0.17, 0.23, 0.27]).values
            df[[f'{i}_diff1_roll_7' for i in main_cols]]=(df_full_before_f[diff1_cols][-7:].values*weights7).mean(axis=0)
            df[[f'{i}_diff2_roll_7' for i in main_cols]]=(df_full_before_f[diff2_cols][-7:].values*weights7).mean(axis=0)
            df[[f'{i}_diff3_roll_7' for i in main_cols]]=(df_full_before_f[diff3_cols][-7:].values*weights7).mean(axis=0)


            weights8=pd.DataFrame([0.03, 0.07, 0.8, 0.12, 0.13, 0.17, 0.23, 0.27]).values
            df[[f'{i}_diff1_roll_8' for i in main_cols]]=(df_full_before_f[diff1_cols][-8:].values*weights8).mean(axis=0)
            df[[f'{i}_diff2_roll_8' for i in main_cols]]=(df_full_before_f[diff2_cols][-8:].values*weights8).mean(axis=0)
            df[[f'{i}_diff3_roll_8' for i in main_cols]]=(df_full_before_f[diff3_cols][-8:].values*weights8).mean(axis=0)


        except:
            pass


        # to get the moving average
        lst_cols=[diff1_cols, diff2_cols, diff1_binary_cols, diff2_binary_cols, main_cols]
        for i, col in zip(range(1, 4), lst_cols):
            for j in range(2, 8):
                try:
                    roll_count=df_full_before_f[col].rolling(i).count().add_suffix(f'diff{i}_roll_count_{j}')[-1:].values
                    roll_sum=df_full_before_f[col].rolling(i).sum().add_suffix(f'diff{i}_roll_sum_{j}')[-1:].value
                    roll_mean=df_full_before_f[col].rolling(i).mean().add_suffix(f'diff{i}_roll_mean_{j}')[-1:].values
                    roll_median=df_full_before_f[col].rolling(i).median().add_suffix(f'diff{i}_roll_median_{j}')[-1:].values

                    # to assign the value for each columns
                    df[roll_count.columns]=roll_count.values
                    df[roll_sum.columns]=roll_sum.values
                    df[roll_mean.columns]=roll_mean.values
                    df[roll_median.columns]=roll_median.values

                except:
                    pass



        # lag features
        try:
            for i in 25:
                #df[[f'{j}_lag_diff3{i}' for j in main_cols]]=df_full_before_f[diff3_cols].shift(i)[-1:].values
                #df[[f'{j}_lag_diff2{i}' for j in main_cols]]=df_full_before_f[diff2_cols].shift(i)[-1:].values
                df[[f'{j}_lag_diff1{i}' for j in main_cols]]=df_full_before_f[diff1_cols].shift(i)[-1:].values

                #df[[f'{j}_lag_diff3_binary{i}' for j in main_cols]]=df_full_before_f[diff3_binary_cols].shift(i)[-1:].values
                #df[[f'{j}_lag_diff2_binary{i}' for j in main_cols]]=df_full_before_f[diff2_binary_cols].shift(i)[-1:].values
                #df[[f'{j}_lag_diff1_binary{i}' for j in main_cols]]=df_full_before_f[diff1_binary_cols].shift(i)[-1:].values

                #df[[f'{j}_lag_main_cols{i}' for j in main_cols]]=df_full_before_f[main_cols].shift(i)[-1:].values
                #df[[f'{j}_lag_main_cols{i}' for j in main_cols]]=df_full_before_f[main_cols].shift(i)[-1:].values
                df[[f'{j}_lag_main_cols{i}' for j in main_cols]]=df_full_before_f[main_cols].shift(i)[-1:].values


                #df[[f'{j}_lag_ratio3{i}' for j in main_cols]]=df_full_before_f[ratio3_cols].shift(i)[-1:].values
                #df[[f'{j}_lag_ratio2{i}' for j in main_cols]]=df_full_before_f[ratio2_cols].shift(i)[-1:].values
                df[[f'{j}_lag_ratio1{i}' for j in main_cols]]=df_full_before_f[ratio1_cols].shift(i)[-1:].values

        except:
            pass


        df['count']=int(df_full_before_f.shape[0])
        return df
    except:
        pass

In [ ]:
train = pd.concat([reader(f) for f in tqdm(train_file)]);print(train.shape)
test = pd.concat([reader(f) for f in tqdm(test_file)]); print(test.shape)
train=train[train['count']!=0]
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [ ]:
train.isna().mean().sort_values()

In [ ]:
train['label']=train['label'].astype(int)
train.corr()['label'].sort_values()[-29:]

In [ ]:
# some columns names of lists
all_diff1_cols=[f'{i}_diff1' for i in main_cols]
all_diff2_cols=[f'{i}_diff2' for i in main_cols]
all_diff3_cols=[f'{i}_diff3' for i in main_cols]
all_diff4_cols=[f'{i}_diff4' for i in main_cols]

all_diff01_cols=[f'{i}_diff1-1' for i in main_cols]
all_diff02_cols=[f'{i}_diff2-1' for i in main_cols]
all_diff03_cols=[f'{i}_diff3-1' for i in main_cols]

label1_cols=[f'{i}_label1' for i in main_cols]
label2_cols=[f'{i}_label2' for i in main_cols]
label3_cols=[f'{i}_label3' for i in main_cols]
label4_cols=[f'{i}_label4' for i in main_cols]

all_diff_sum=[f'{i}_all_diff_sum' for i in main_cols]
all_diff_binary=[f'{i}_all_diff_binary_sum' for i in main_cols]


In [ ]:
def get_all_rows(f):
            df = pd.read_csv(f)

            # to put id
            df['id']=f.split('/')[-1].split('.')[0]

            # to get fault status
            df['fualty_stuts']=(df['fault_duration']>0).astype(int)


            # to get diff of t+1 -t+1 values of main cols
            df[all_diff1_cols]=df[main_cols].diff(1).values
            df[all_diff2_cols]=df[all_diff1_cols].diff(1).values
            df[all_diff3_cols]=df[all_diff2_cols].diff(1).values


            # to get lag values
            df[all_diff01_cols]=df[all_diff1_cols].shift(1).values
            df[all_diff02_cols]=df[all_diff2_cols].shift(1).values
            df[all_diff03_cols]=df[all_diff3_cols].shift(1).values


            return df

In [ ]:
train_all=pd.concat([get_all_rows(f) for f in tqdm(train_file)])
test_all=pd.concat([get_all_rows(f) for f in tqdm(test_file)])

In [ ]:
train_all['train']=1
test_all['train']=0
all_data=pd.concat([train_all, test_all], axis=0)

In [ ]:
# to put binary value on diff values
all_data[label1_cols]=(all_data[all_diff01_cols]<0).astype(np.int16)
all_data[label2_cols]=(all_data[all_diff02_cols]<0).astype(np.int16)
all_data[label3_cols]=(all_data[all_diff03_cols]<0).astype(np.int16)

#to sum up of binary columns
all_data[all_diff_binary]=all_data[label1_cols].values+all_data[label2_cols].values+all_data[label3_cols].values
train[all_diff_binary]=(train[diff1_binary_cols].values+train[diff2_binary_cols].values+train[diff3_binary_cols].values).astype(int)
test[all_diff_binary]=(test[diff1_binary_cols].values+test[diff2_binary_cols].values+test[diff3_binary_cols].values).astype(int)

In [ ]:
# to get outlier value for diff columns
q3=all_data[all_diff1_cols].quantile(0.75)
q1=all_data[all_diff1_cols].quantile(0.25)

IQR=q3-q1
lower=q1-1.5 *IQR
upper=q1+1.5 *IQR

all_data=all_data.fillna(0)

all_outlier=((all_data[all_diff1_cols]<lower.values).astype(int)+ (all_data[all_diff1_cols]>upper.values).astype(int)).add_suffix('_outlier_values')
all_data[all_outlier.columns]=all_outlier.values

In [ ]:
all_outlier=((all_data[all_diff1_cols]<lower.values).astype(int)+ (all_data[all_diff1_cols]>upper.values).astype(int)).add_suffix('_outlier_values')
all_data[all_outlier.columns]=all_outlier.values

In [ ]:
cols_all_train=[col for col in train_all.columns if col not in ['Unnamed: 0', 'NE ID', 'endTime', 'id']]

In [ ]:
train_all.isna().mean().sort_values()

### 3. Feature engineering

In [ ]:
# to create columns names
agg_cols=['mean', 'size', 'sum', 'min', 'max']
def get_columns(cols, agg_lst=agg_cols):
    new_cols=[]
    lst_suffix=agg_lst
    for col in cols:
        if col not in ['id', 'NE ID']:
            for suffix in lst_suffix:
                new_cols.append(col+f'_{suffix}')
        else:
            new_cols.append(col)
    return new_cols


# list of agg fun
agg_lsts=['mean', 'size', 'sum', 'min', 'max']

In [ ]:
# to replace some relation values which are less than 0 with 0
train.loc[train['relation']<0, 'relation']=0
test.loc[test['relation']<0, 'relation']=0
test.loc[test['relation']<0, 'relation']=0

# to get id block
train['ID_1']=train['NE ID'].str.split('-', expand=True)[0]
all_data['ID_1']=all_data['NE ID'].str.split('-', expand=True)[0]
test['ID_1']=test['NE ID'].str.split('-', expand=True)[0]


# to get data features
train=date_features(train)
test=date_features(test)
all_data=date_features(all_data)


# to create categorical value of fault_duration
train['fault_duration_min']=train['fault_duration']//60
train['fault_duration_min/10']=(train['fault_duration']//(60*10)).astype(np.int16)

test['fault_duration_min']=test['fault_duration']//60
test['fault_duration_min/10']=(test['fault_duration']//(60*10)).astype(np.int16)

all_data['fault_duration_min']=all_data['fault_duration']//60
all_data['fault_duration_min/10']=(all_data['fault_duration']//(60*10)).astype(np.int16)


# to create categorical value of relation
train['relation_int']=((train['relation']*10)//4).astype(np.int16)
test['relation_int']=((test['relation']*10)//4).astype(np.int16)
all_data['relation_int']=((all_data['relation']*10)//4).astype(np.int16)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

label_en=LabelEncoder()
k_mean=KMeans(n_clusters=5)

id_col=pd.concat([train['ID_1'], test['ID_1']], axis=0)
label_en.fit(id_col)


test['count_ID_1']=test['ID_1'].map(test['ID_1'].value_counts().to_dict())
train['count_ID_1']=train['ID_1'].map(train[['ID_1']].value_counts().to_dict())


k_mean.fit(pd.concat([train[['relation', 'fault_duration']], test[['relation', 'fault_duration']]], axis=0))
train['cluster_id']=k_mean.predict(train[['relation', 'fault_duration']])
test['cluster_id']=k_mean.predict(test[['relation', 'fault_duration']])
all_data['cluster_id']=k_mean.predict(all_data[['relation', 'fault_duration']])

In [ ]:
# to join to two or more columns
def com_col(df):
    df['daydayofweekhour_cluster_id']=df[['daydayofweekhour', 'cluster_id']].astype(str).apply(''.join, axis=1).astype(int)

    df['hour//4_day']=df[['hour//4', 'day']].astype(str).apply(''.join, axis=1).astype(int)
    df['hour//2_day']=df[['hour//2', 'day']].astype(str).apply(''.join, axis=1).astype(int)
    df['hour//6_day']=df[['hour//6', 'day']].astype(str).apply(''.join, axis=1).astype(int)
    df['weekend/day']=df[['weekend', 'day']].astype(str).apply(''.join, axis=1).astype(int)
    df['weekend_hour']=df[['hour', 'weekend']].astype(str).apply(''.join, axis=1).astype(int)

    df['fault_duration_hour']=df[['fault_duration_min/10', 'hour']].astype(str).apply(''.join, axis=1).astype(int)
    df['fault_duration_dayofweek']=df[['fault_duration_min/10', 'dayofweek']].astype(str).apply(''.join, axis=1).astype(int)
    df['fault_duration_dayofweekhour']=df[['fault_duration_min/10','dayofweek', 'hour']].astype(str).apply(''.join, axis=1).astype(int)

    df['relation_hour']=df[['relation_int', 'hour']].astype(str).apply(''.join, axis=1).astype(int)
    df['relation_weekday']=df[['relation_int', 'dayofweek']].astype(str).apply(''.join, axis=1).astype(int)
    df['relation_dayofweekhour']=df[['relation_int', 'dayofweek', 'hour']].astype(str).apply(''.join, axis=1).astype(int)


    df['relation_fault']=df[['relation_int', 'fault_duration_min/10']].astype(str).apply(''.join, axis=1).astype(int)
    df['relation_fault_hour']=df[['relation_fault', 'hour']].astype(str).apply(''.join, axis=1).astype(int)
    df['relation_fault_weekday']=df[['relation_fault', 'dayofweek']].astype(str).apply(''.join, axis=1).astype(int)
    df['relation_fault_weekday']=df[['relation_fault', 'dayofweek', 'hour']].astype(str).apply(''.join, axis=1).astype(int)

    df['dayofweek_data_rate_all_diff_binary_sum']=df[['dayofweek', 'data_rate_all_diff_binary_sum']].fillna(99).astype(int).astype(str).apply(''.join, axis=1).astype(int)
    df['dayofweek_data_rate_all_diff_binary_sum']=df[['hour', 'data_rate_all_diff_binary_sum']].fillna(99).astype(int).astype(str).apply(''.join, axis=1).astype(int)
    df['dayofweekhour_data_rate_all_diff_binary_sum']=df[['dayofweekhour', 'data_rate_all_diff_binary_sum']].fillna(99).astype(int).astype(str).apply(''.join, axis=1).astype(int)
    df['relation_fault_data_rate_all_diff_binary_sum']=df[['relation_fault', 'data_rate_all_diff_binary_sum']].fillna(99).astype(int).astype(str).apply(''.join, axis=1).astype(int)
    df['fault_duration_min/10_data_rate_all_diff_binary_sum']=df[['fault_duration_min/10', 'data_rate_all_diff_binary_sum']].fillna(99).astype(int).astype(str).apply(''.join, axis=1).astype(int)
    df['relation_int_data_rate_all_diff_binary_sum']=df[['relation_int', 'data_rate_all_diff_binary_sum']].fillna(99).astype(int).astype(str).apply(''.join, axis=1).astype(int)


    df['relation_int_dayofweekhour_data_rate_all_diff_binary_sum']=df[['relation_fault','dayofweekhour' ,'data_rate_all_diff_binary_sum']].astype(int).astype(str).apply(''.join, axis=1).astype(int)



    return df



In [ ]:
train=com_col(train)
test=com_col(test)
all_data=com_col(all_data)

all_data_befor_f=all_data[all_data['fualty_stuts']==0]
all_data_after_f=all_data[all_data['fualty_stuts']>0]

In [ ]:
train_df=train.copy()
test_df=test.copy()

In [ ]:
# to mean encode some features

drop_cols=[f'{i}_ratio1+1' for i in main_cols]+[f'{i}_ratio2+1' for i in main_cols]+[f'{i}_ratio3+1' for i in main_cols]+[f'{i}_diff1+1' for i in main_cols]+[f'{i}_diff3+1' for i in main_cols]+[f'{i}_diff2+1' for i in main_cols]
cat_cols=['Unnamed: 0', 'y', 'y_hat', 'id', 'ID','ID_1', 'NE ID', 'endTime', 'label', 'data_rate_ratio','data_rate_dif+1', 'data_rate_diff', 'ID_1_gbyid']
drop_cols=drop_cols+cat_cols


# to get corrrelations
#train['label']=train['label'].astype(int)
train_cor_cols=train.corr()['label'].sort_values()

# to slice to top correlated cols
top_cor_cols=list(train_cor_cols[train_cor_cols>.1].index)+list(train_cor_cols[train_cor_cols<-.1].index)
top_cor_cols=[col for col in top_cor_cols if col not in drop_cols ]

In [ ]:
# to group and merge features
def group_cat(df, cols, tar_cols, suf, train=train):
    """
    df : data this may be train or test
    cols: which columns we want to group the data
    tar_cols: which columns we want to get mean values
    suf: suffix name

    train: train data

    > return data that added new features that averaged by cols
    """

    mean_t=train.groupby(cols)[tar_cols].mean().reset_index().add_suffix(f'_mean_target_{cols}_{suf}')
    df=df.merge(mean_t, how='left',left_on=[cols], right_on=[f'{cols}_mean_target_{cols}_{suf}'])
    return df

In [ ]:
cols_tar=['day',  'dayofweekhour', 'daydayofweekhour',
          'fault_duration_hour', 'fault_duration_dayofweek',
          'relation_hour', 'relation_weekday', 'relation_fault_weekday',
          'relation_fault', 'relation_fault_hour',
         'weekend_hour', 'weekend/day', 'hour//4_day', 'hour//2_day', 'hour//6_day']

# to mean encode all from all train rows
for i in cols_tar:
    train_df=group_cat(train_df, i, top_cor_cols, 'all_rows_train',  train=train)
    test_df=group_cat(test_df, i, top_cor_cols,'all_rows_train',  train=train)


In [ ]:
cols_tar=['day', 'dayofweek', 'hour',
          'fault_duration_hour',
         'weekend_hour', 'weekend/day', 'hour//4_day', 'hour//2_day', 'hour//6_day']



# to mean encode from label 1 and 0
for i in cols_tar:
    train_df=group_cat(train_df, i, top_cor_cols, 'all_rows_train1', train=train[train['label']==1])
    test_df=group_cat(test_df, i, top_cor_cols,'all_rows_train1', train=train[train['label']==1])

    train_df=group_cat(train_df, i, top_cor_cols, 'all_rows_train0', train=train[train['label']==0])
    test_df=group_cat(test_df, i, top_cor_cols, 'all_rows_train0', train=train[train['label']==0])

In [ ]:
# some preprocess data
def pre_data(df1,df2, gp_cols,filter_cols,suf, agg_lst=agg_lsts):

    """ input:
         df1 : all rows that may include from train and test
              all data

         df2 : train and test files that on targeted rows  that mean train is 7212 and test 1992

         gp_cols :  data group by this columns
         filter_cols: slice columns
         agg_lst : agg_list
    return: df that merged with mean encode features
    """

    # to get columns name
    cols=get_columns(cols=filter_cols)

    # to join columns
    union_col=''.join(gp_cols)
    cols=[f'{i}_{union_col}_{suf}' for i in cols]

    gp_df=df1.groupby(gp_cols)[filter_cols].agg(agg_lsts)
    gp_df.columns=cols


    gp_df=gp_df.reset_index()


    df=df2.merge(gp_df, how='left', on=gp_cols)

    return df




In [ ]:
# agg function lists
agg_cols=['relation_fault', 'relation_fault_hour',
          'relation_fault_weekday', 'relation_fault_weekday',
          'fault_duration_hour','fault_duration_dayofweek',
          'fault_duration_dayofweekhour',
          'relation_hour','relation_weekday','relation_weekday',
          'relation_dayofweekhour',
          'dayofweek_data_rate_all_diff_binary_sum',
          'dayofweek_data_rate_all_diff_binary_sum',
          'dayofweekhour_data_rate_all_diff_binary_sum',
          'relation_fault_data_rate_all_diff_binary_sum',
           'fault_duration_min/10_data_rate_all_diff_binary_sum',
          'relation_int_data_rate_all_diff_binary_sum',
          'relation_int_dayofweekhour_data_rate_all_diff_binary_sum',
          'daydayofweekhour_cluster_id']

In [ ]:
# to agg and merge with different columns
def merge_df(df1, df2, cols1, filter_cols, suf):
    df=pre_data(df1, df2, [cols1], filter_cols, suf)
    df=pre_data(df1, df, [cols1, 'hour'], filter_cols, suf)
    df=pre_data(df1, df, [cols1, 'dayofweek'], filter_cols, suf)
    df=pre_data(df1, df, [cols1, 'dayofweekhour'], filter_cols, suf)
    df=pre_data(df1, df, [cols1, 'daydayofweekhour'], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[0]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[1]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[2]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[3]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[4]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[5]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[6]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[7]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[8]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[9]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[10]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[10]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[11]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[12]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[13]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[14]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[15]], filter_cols, suf)
    df=pre_data(df1, df, [cols1, agg_cols[16]], filter_cols, suf)
    return df

In [ ]:
#to mean encode and merge from train and test file
train_test_df=pd.concat([train, test],axis=0)
id_df=train_test_df[['ID_1', 'NE ID','id']]

train_df=merge_df(train_test_df, train_df, 'NE ID', top_cor_cols, 'train_mean_en_top_cols')
train_df=merge_df(train_test_df, train_df, 'ID_1', top_cor_cols, 'train_mean_en_top_cols')

test_df=merge_df(train_test_df, test_df, 'NE ID', top_cor_cols, 'train_mean_en_top_cols')
test_df=merge_df(train_test_df, test_df, 'ID_1', top_cor_cols, 'train_mean_en_top_cols')



In [ ]:
# to mean encode with all rows
train_df=merge_df(all_data_befor_f, train_df, 'NE ID',  cols_all_train, '_mean_en_all_rows')
train_df=merge_df(all_data_befor_f, train_df, 'ID_1', cols_all_train, '_mean_en_all_rows')
train_df=merge_df(all_data_befor_f, train_df, 'id', cols_all_train, '_mean_en_all_rows')


test_df=merge_df(all_data_befor_f, test_df, 'NE ID', cols_all_train, '_mean_en_all_rows')
test_df=merge_df(all_data_befor_f, test_df, 'ID_1', cols_all_train, '_mean_en_all_rows')
test_df=merge_df(all_data_befor_f, test_df, 'id', cols_all_train, '_mean_en_all_rows')

In [ ]:
all_data_befor_f0=all_data_befor_f[all_data_befor_f['data_rate_diff1']>=0]
all_data_befor_f1=all_data_befor_f[all_data_befor_f['data_rate_diff1']<0]

In [ ]:
# to mean encode and  from all rows which are diff of data rate is greater than 0
train_df=merge_df(all_data_befor_f0, train_df, 'NE ID',  all_diff01_cols, '_mean_en_all_rows_label0')
train_df=merge_df(all_data_befor_f0, train_df, 'ID_1', all_diff01_cols, '_mean_en_all_rows_label0')
#train_df=merge_df(all_data_befor_f0, train_df, 'id ', all_diff01_cols, '_mean_en_all_rows_label0')


test_df=merge_df(all_data_befor_f0, test_df, 'NE ID', all_diff01_cols, '_mean_en_all_rows_label0')
test_df=merge_df(all_data_befor_f0, test_df, 'ID_1', all_diff01_cols, '_mean_en_all_rows_label0')
#test_df=merge_df(all_data_befor_f0, test_df, 'id', all_diff01_cols, '_mean_en_all_rows_label0')

In [ ]:
# to mean encode and  from all rows which are diff of data rate is less than 0
train_df=merge_df(all_data_befor_f1, train_df, 'NE ID',  all_diff01_cols, '_mean_en_all_rows_label1')
train_df=merge_df(all_data_befor_f1, train_df, 'ID_1', all_diff01_cols, '_mean_en_all_rows_label1')
#train_df=merge_df(all_data_befor_f1, train_df, 'id', all_diff01_cols, '_mean_en_all_rows_label1')


test_df=merge_df(all_data_befor_f1, test_df, 'NE ID', all_diff01_cols, '_mean_en_all_rows_label1')
test_df=merge_df(all_data_befor_f1, test_df, 'ID_1', all_diff01_cols, '_mean_en_all_rows_label1')
#test_df=merge_df(all_data_befor_f1, test_df, 'id', all_diff01_cols, '_mean_en_all_rows_label1')

In [ ]:
# to merge with time related features
def merge_date_feat(df1, df2, suf):
    df=pre_data(df1,df2, 'hour',main_cols, suf,  agg_lst=agg_lsts)
    df=pre_data(df1,df, 'dayofweek',main_cols, suf,agg_lst=agg_lsts)
    df=pre_data(df1, df, 'dayofweekhour',main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, 'day',main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, 'daydayofweekhour',main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[0],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[1],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[2],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[3],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[4],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[5],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[6],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[7],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[8],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[9],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[10],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[11],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[12],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[13],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[14],main_cols, suf,agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[15],main_cols,suf, agg_lst=agg_lsts)
    df=pre_data(df1, df, agg_cols[16],main_cols,suf, agg_lst=agg_lsts)


    return df

In [ ]:
all_data_after_f=all_data_after_f[all_data_after_f['train']==1]
all_data_after_f1=all_data_after_f[all_data_after_f['data_rate_diff1']<0]
all_data_after_f0=all_data_after_f[all_data_after_f['data_rate_diff1']>=0]

In [ ]:
train_df=merge_date_feat(all_data_after_f, train_df, '_mean_target_en_all_rows')
test_df=merge_date_feat(all_data_after_f, test_df, '_mean_target_en_all_rows')

train_df=merge_date_feat(all_data_after_f0, train_df, '_mean_target_en_all_rows_label0')
test_df=merge_date_feat(all_data_after_f0, test_df, '_mean_target_en_all_rows_label0')

train_df=merge_date_feat(all_data_after_f1, train_df, '_mean_target_en_all_rows_label1')
test_df=merge_date_feat(all_data_after_f1, test_df, '_mean_target_en_all_rows_label1')

In [ ]:
def encode_features(df,cols, train_df=train_df):

    dict_count_hour=train_df[train_df['label']==1]['hour'].value_counts().to_dict()
    dict_mean_hour=train_df.groupby('hour')['label'].mean().to_dict()

    dict_count_day=train_df[train_df['label']==1]['dayofweek'].value_counts().to_dict()
    dict_mean_day=train_df.groupby('dayofweek')['label'].mean().to_dict()

    dict_count_dayofmonth=train_df[train_df['label']==1]['day'].value_counts().to_dict()
    dict_mean_dayofmonth=train_df.groupby('day')['label'].mean().to_dict()

    df[f'mean_encode_hour_{cols}']=df['hour'].map(dict_mean_hour)
    df[f'mean_encode_day_{cols}']=df['dayofweek'].map(dict_mean_day)
    df[f'mean_count_dayofmonth_{cols}']=df['day'].map(dict_count_dayofmonth)
    df[f'mean_encode_dayofmonth_{cols}']=df['day'].map(dict_mean_dayofmonth)


    df[f'hour_mean_data_rate_diff_{cols}']=df['hour'].map(train_df.groupby('hour')[cols].mean().to_dict())
    df[f'hour_max_data_rate_diff_{cols}']=df['hour'].map(train_df.groupby('hour')[cols].max().to_dict())
    df[f'hour_min_data_rate_diff_{cols}']=df['hour'].map(train_df.groupby('hour')[cols].min().to_dict())
    df[f'hour_median_data_rate_diff_{cols}']=df['hour'].map(train_df.groupby('hour')[cols].median().to_dict())
    df[f'hour_q1_data_rate_diff_{cols}']=df['hour'].map(train_df.groupby('hour')[cols].quantile(0.25).to_dict())
    df[f'hour_q2_data_rate_diff_{cols}']=df['hour'].map(train_df.groupby('hour')[cols].quantile(0.5).to_dict())
    df[f'hour_q3_data_rate_diff_{cols}']=df['hour'].map(train_df.groupby('hour')[cols].quantile(0.75).to_dict())
    df[f'hour_q4_data_rate_diff_{cols}']=df['hour'].map(train_df.groupby('hour')[cols].quantile(0.90).to_dict())


    df[f'day_mean_data_rate_diff_{cols}']=df['dayofweek'].map(train_df.groupby('dayofweek')[cols].mean().to_dict())
    df[f'day_max_data_rate_diff_{cols}']=df['dayofweek'].map(train_df.groupby('dayofweek')[cols].max().to_dict())
    df[f'day_min_data_rate_diff_{cols}']=df['dayofweek'].map(train_df.groupby('dayofweek')[cols].min().to_dict())
    df[f'day_median_data_rate_diff_{cols}']=df['dayofweek'].map(train_df.groupby('dayofweek')[cols].median().to_dict())
    df[f'day_q1_data_rate_diff_{cols}']=df['dayofweek'].map(train_df.groupby('dayofweek')[cols].quantile(0.25).to_dict())
    df[f'day_q2_data_rate_diff_{cols}']=df['dayofweek'].map(train_df.groupby('dayofweek')[cols].quantile(0.5).to_dict())
    df[f'day_q3_data_rate_diff_{cols}']=df['dayofweek'].map(train_df.groupby('dayofweek')[cols].quantile(0.75).to_dict())
    df[f'dayhour_mean_data_rate_diff_{cols}']=df['dayofweekhour'].map(train_df.groupby('dayofweekhour')[cols].mean().to_dict())
    df[f'dayhour_max_data_rate_diff_{cols}']=df['dayofweekhour'].map(train_df.groupby('dayofweekhour')[cols].max().to_dict())
    df[f'dayhour_min_data_rate_diff_{cols}']=df['dayofweekhour'].map(train_df.groupby('dayofweekhour')[cols].min().to_dict())
    df[f'dayhour_median_data_rate_diff_{cols}']=df['dayofweekhour'].map(train_df.groupby('dayofweekhour')[cols].median().to_dict())
    df[f'dayhour_q1_data_rate_diff_{cols}']=df['dayofweekhour'].map(train_df.groupby('dayofweekhour')[cols].quantile(0.25).to_dict())
    df[f'dayhour_q2_data_rate_diff_{cols}']=df['dayofweekhour'].map(train_df.groupby('dayofweekhour')[cols].quantile(0.5).to_dict())
    df[f'dayhour_q3_data_rate_diff_{cols}']=df['dayofweekhour'].map(train_df.groupby('dayofweekhour')[cols].quantile(0.75).to_dict())
    df[f'dayhour_q4_data_rate_diff_{cols}']=df['dayofweekhour'].map(train_df.groupby('dayofweekhour')[cols].quantile(0.90).to_dict())

    return df


In [ ]:
diff_cols=[f'{i}_diff1+1' for i in main_cols]
for col in diff_cols:
    print(col)
    train_df=encode_features(train_df,col, train_df=train_df)
    test_df=encode_features(test_df,col, train_df=train_df)

diff_cols=[f'{i}_diff2+1' for i in main_cols]
for col in diff_cols:
    print(col)
    train_df=encode_features(train_df,col, train_df=train_df)
    test_df=encode_features(test_df,col, train_df=train_df)


In [ ]:
def get_cluster(df, n_clusters=3, model_return=True):
    from sklearn.cluster import KMeans
    model_cluster=KMeans(n_clusters=n_clusters)

    #df=pd.concat([train_set, test_set], axis=1)
    try:
        model_cluster.fit(df.fillna(0))
    except:
        df=df.astype(np.float32)
        df=df.replace([-np.inf, np.inf], np.nan)
        df=df.fillna(0)
        model_cluster.fit(df.fillna(0))
    if model_return:
        return model_cluster

In [ ]:
dict_k_model={}
k_cols=[diff1_cols, diff2_cols, diff3_cols, diff1_cols+diff2_cols+diff3_cols,  ratio1_cols,top_cor_cols,
       train_df.select_dtypes('number').columns.to_list()]
k_model=['diff1', 'diff2', 'diff3_cols', 'diff1_cols+diff2_cols+diff3_cols',  'ratio1_cols']
for col, k_name in zip(k_cols, k_model):
    dict_k_model[k_name]=get_cluster(train[col])

In [ ]:
def cluster_predict(df):
    for col, k_name in zip(k_cols, k_model):
        try:
            df[f'clustered_{k_name}']=dict_k_model[k_name].predict(df[col].fillna(0))
        except:
            df[col]=df[col].astype(np.float32)
            df=df.replace([-np.inf, np.inf], np.nan)
            df=df.fillna(0)
            df[f'clustered_{k_name}']=dict_k_model[k_name].predict(df[col].fillna(0))
    return df



In [ ]:
train_df=cluster_predict(train_df)
test_df=cluster_predict(test_df)

In [ ]:
print(train_df.shape)
test_df.shape

In [ ]:
drop_cols=[f'{i}_ratio1+1' for i in main_cols]+[f'{i}_ratio2+1' for i in main_cols]+[f'{i}_ratio3+1' for i in main_cols]+[f'{i}_diff1+1' for i in main_cols]+[f'{i}_diff3+1' for i in main_cols]+[f'{i}_diff2+1' for i in main_cols]
cat_cols=['Unnamed: 0', 'y', 'y_hat', 'id', 'ID','ID_1', 'NE ID', 'endTime', 'label', 'data_rate_ratio','data_rate_dif+1', 'data_rate_diff', 'ID_1_gbyid']
drop_cols=drop_cols+cat_cols
features=[col for col in train_df.columns if col not in drop_cols]
label='label'
train_balanced=train_df
X=train_balanced[features]
y=train_balanced[label]
test_x=test_df[features]
X.shape

In [ ]:
groups=train_balanced['ID_1'].str.replace('B', '').astype(np.float32)

In [ ]:
model=LGBMClassifier(silent=True, colsample_bytree=0.7)
model.fit(X, y)
try:
    feat_save=pd.DataFrame({ 'inpo':model.feature_importances_, 'col_name':X.columns}).sort_values(by=['inpo'],ascending=False)
except:
    feat_save=pd.Series(model.feature_importances_, X.columns).sort_values(ascending=False)

feat_impo=pd.Series(model.feature_importances_, X.columns).sort_values(ascending=False)
selected_feat=feat_impo[:50].index

In [ ]:
feat_save.to_csv('feat_impo.csv', index=False)
feat_impo[:30]


In [ ]:
def lgb_f1_score(y_hat, y):
    y_true = (y>=.5).astype(int)
    y_hat = (y_hat>=0.5).astype(int)
    return 'f1', f1_score(y_true, y_hat), True

### 4. Building model

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold, KFold
from sklearn.metrics import f1_score
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
def train_model(X, y, x_test, feat=X.columns, model=None, n_fold=2, model_name=None,  classifier=False, final=False):

    oof=np.zeros(X.shape[0])
    ouf=[]
    test_preds=[]
    fold= StratifiedKFold(n_splits=n_fold)
    step=0
    if final:
        X['count']=train_balanced['count']
        x_test['count']=test_df['count']

    for train_index, val_index in fold.split(train_balanced[diff1_cols], y):
        step+=1
        print(f'fold is {step}')

        if not final:
            X_train, y_train=X[feat].iloc[train_index], y.iloc[train_index]
            X_valid, y_valid=X[feat].iloc[val_index], y.iloc[val_index]
        if final:
            X_train, y_train=X.iloc[train_index], y.iloc[train_index]
            X_valid, y_valid=X.iloc[val_index], y.iloc[val_index]
        #model
        model=model

        if classifier:
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid),verbose=250,
              #eval_metric=lgb_f1_score,
              #early_stopping_rounds=500,
                
                                    )

        if classifier:
            if model_name=='lgb':
                val_preds=model.predict_proba(X_valid, num_iteration=model.best_iteration_)[:, 1]
                test_preds.append(model.predict_proba(x_test[X_train.columns], num_iteration=model.best_iteration_)[:, 1])
            if model_name=='cat':
                val_preds=model.predict_proba(X_valid)[:, 1]
                test_preds.append(model.predict_proba(x_test[X_train.columns])[:, 1])

        oof[val_index]=val_preds

        score=f1_score((y_valid>=.5).astype(int), (val_preds>=.5).astype(int))
        print(f'fold f1_score is {score}')

    print(f'total f1_score is {f1_score((y>=.5).astype(int), (oof>=.5).astype(int))}')
    predictions=np.mean(test_preds, axis=0)
    return oof, predictions




In [ ]:
from sklearn.metrics import classification_report

In [ ]:
model=LGBMClassifier(n_estimators=5000, learning_rate=0.005, random_seed=41,
                    #class_weight='balanced',
                    num_leaves=35,
                    feature_fraction=0.9,
                    max_depth=15,
    
                    early_stopping_rounds=500

                       )

oof_lgb, pred_lgb=train_model(X, train_balanced['label'],  test_x, feat=feat_impo[:50].index, model=model,n_fold=10,  model_name='lgb', classifier=True)

pd.Series(pred_lgb).to_csv('pred_lgb.csv', index=False)

In [ ]:
print(f'total f1_score is {f1_score(y, (oof_lgb>=.35).astype(int))}')
print('lightgmb:')
print(classification_report(y, (oof_lgb>=.35).astype(int)))


In [ ]:
model=LGBMClassifier(n_estimators=5000, learning_rate=0.01, random_seed=41,
                    #class_weight='balanced',
                    num_leaves=35,
                    feature_fraction=0.6,
                    max_depth=15,
                    #verbose=250,
                    early_stopping_rounds=500
                       )

oof_lgb2, pred_lgb2=train_model(X, train_balanced['label'],  test_x, feat=feat_impo[:100].index, model=model,n_fold=10, classifier=True, model_name='lgb')
pd.Series(pred_lgb2).to_csv('pred_lgb2.csv', index=False)

In [ ]:
print(f'total f1_score is {f1_score(y, (oof_lgb2>=.35).astype(int))}')
print('lightgmb2:')
print(classification_report(y, (oof_lgb2>=.35).astype(int)))


In [ ]:
model=CatBoostClassifier(n_estimators=5000, learning_rate=0.01, random_seed=41,
                    auto_class_weights='Balanced',
                    max_depth=5,
                    bagging_temperature=0.7,
                    use_best_model=True,
                    early_stopping_rounds=500
                    
                       )

oof_cat, pred_cat=train_model(X, train_balanced['label'], test_x, feat=feat_impo[:80].index, model=model,n_fold=10, classifier=True, model_name='cat')
pd.Series(pred_cat).to_csv('pred_cat.csv', index=False)

In [ ]:
print(f'total f1_score is {f1_score(y, (oof_cat>=.35).astype(int))}')
print('catboost2:')
print(classification_report(y, (oof_cat>=.35).astype(int)))


In [ ]:
oof_X=pd.DataFrame({'cat':oof_cat, 'lgb':oof_lgb, 'lgb2':oof_lgb2})
oof_test=pd.DataFrame({'cat':pred_cat, 'lgb':pred_lgb, 'lgb2':pred_lgb2})

oof_X['mean']=oof_X.mean(axis=1)
oof_test['mean']=oof_test.mean(axis=1)


In [ ]:
model=CatBoostClassifier(n_estimators=5000, learning_rate=0.005, random_seed=41,
                    auto_class_weights='Balanced',
                    verbose=250,
                    early_stopping_rounds=500,
                       )

oof_cat2, pred_cat2=train_model(oof_X, train_balanced['label'], oof_test, model=model,n_fold=10, classifier=True, model_name='cat', final=True)
pd.Series(pred_cat2).to_csv('pred_cat2.csv', index=False)

In [ ]:
print(f'total f1_score is {f1_score(y, (oof_cat2>=.35).astype(int))}')
print('catboost final model:')
print(classification_report(y, (oof_cat2>=.35).astype(int)))


In [ ]:
pred=(pred_cat2>.35).astype(int)

In [ ]:
ss=pd.read_csv(path_sub)
n_sub=1
ss['data_rate_t+1_trend']=pred
ss['ID']=test['id'].values
ss.to_csv(f'sub_.csv', index=False)
ss['data_rate_t+1_trend'].value_counts()